## Installing Necessary Libraries

In [1]:
!pip install openai pinecone-client[grpc] datasets tiktoken langchain pandas
!pip install --upgrade gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Importing Necessary Libraries
import pandas as pd

import openai
#from openai.embeddings_utils import get_embedding

from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.agents import Tool
from langchain.agents import initialize_agent

from tqdm.auto import tqdm
from uuid import uuid4
import pinecone
from pinecone import Pinecone
import gradio as gr

In [5]:
# Loading Dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cleaned_amazon_QA.csv')
df.drop(columns=["Unnamed: 0"], inplace=True)
df

,id_num,Question,Answer
0,1,What does the Hornby 2014 Catalogue contain?,The Hornby 2014 Catalogue details a wide range...
1,2,Are there any customer reviews for the Hornby ...,"Yes, customers have provided positive feedback..."
2,3,Can the Hornby 2014 Catalogue be used as a ref...,Absolutely! Customers have praised the catalog...
3,4,Is the Hornby 2014 Catalogue suitable for coll...,Definitely! Many collectors appreciate the glo...
4,5,What age group is the Hornby 2014 Catalogue su...,The catalogue is recommended for ages 6 and up...
...,...,...,...
441,442,How does the Optic ID work for the Vision Pro?,"Optic ID data is encrypted, never leaves your ..."
442,443,What security is in place for the Vision Pro?,Use Optic ID to authorize purchases and unlock...
443,444,What chips are used in the Vision Pro?,A unique dual‑chip design enables the spatial ...
444,445,What kind of chip is used for input and sensor...,And the brand-new R1 chip is specifically dedi...


In [6]:
# Accessing OpenAI API key

#Paste your OpenAI API key below
openai_api_key = "Paste your OpenAI API KEY"
# Using Embedding model "text-embedding-3-small"
model_name = 'text-embedding-3-small'
embed = OpenAIEmbeddings(
    model = model_name,
    openai_api_key=openai_api_key,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
#Accesing Pinecone Api key for vector indexing
pc = Pinecone(api_key="de264974-3320-41d8-b53e-819ddacf5f4c")
index = pc.Index("divine-d")

In [10]:
# Uploading data to pinecone
batch_size = 5

texts = []
metadatas = []

for i in tqdm(range(0, len(df), batch_size)):
    i_end = min(len(df), i+batch_size)
    batch = df.iloc[i:i_end]

    metadatas = [{
        'text': str(record['Question']),
        'text': str(record['Answer'])
    } for j, record in batch.iterrows()]
    documents = batch['Answer']
    batch['Answer'] = batch['Answer'].fillna('')
    documents = batch['Answer'].astype(str)
    embeds = embed.embed_documents(documents)
    ids = batch['id_num'].astype(str)
    index.upsert(vectors=zip(ids, embeds, metadatas))

  0%|          | 0/90 [00:00<?, ?it/s]

<ipython-input-10-82727f8b7daf>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['Answer'] = batch['Answer'].fillna('')
<ipython-input-10-82727f8b7daf>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['Answer'] = batch['Answer'].fillna('')
<ipython-input-10-82727f8b7daf>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [8]:
#Seeing if data has been uploaded successfully
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 446}},
 'total_vector_count': 446}

In [9]:
# Creating a vestor store from pinecone for retreival model to access
from langchain.vectorstores import Pinecone
text_field = "text"
#index = pinecone.Index(index_name)
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 0.3.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [10]:
from sqlalchemy.sql.expression import true
llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)
# creating chat buffer to save previous chats
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# Creating QA type model which accesses Pinecone vector data
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
# Defining tool which answers questions based on training data and answers general questions based on ChatGPT answers
tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        )
    )
]

In [12]:
# Initializing agent
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=False,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [14]:
!chmod +x /usr/local/lib/python3.10/dist-packages/gradio/frpc_linux_amd64_v0.2

In [15]:
# Defining Gradio Interface for user interface and model connection
with gr.Blocks() as demo:
  chatbot = gr.Chatbot(value=[[None, "Hi, my name is Divine-D. How Can I help you?"]])
  msg = gr.components.Textbox()
  clear = gr.ClearButton([msg, chatbot])

  def respond(message, chat_history):
    bot_message = agent.run(message)
    print(bot_message)
    chat_history.append((message, bot_message))
    return "", chat_history

  msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c05836eed211e08526.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


I found some popular railway toys that you might like: model train sets, wooden train sets, LEGO train sets, and electric train sets. These are great options for railway enthusiasts of all ages.
I found some popular railway toys that you might like: model train sets, wooden train sets, LEGO train sets, and electric train sets. These are great options for railway enthusiasts of all ages.
The current Prime Minister of India is Narendra Modi.
The President of India is Ram Nath Kovind.
The popular railway toys that you might like are model train sets, wooden train sets, LEGO train sets, and electric train sets. These are great options for railway enthusiasts of all ages.
The Prime Minister of India is Narendra Modi.
The popular railway toys that you might like are model train sets, wooden train sets, LEGO train sets, and electric train sets. These are great options for railway enthusiasts of all ages.
The current Prime Minister of India is Narendra Modi.
Keyboard interruption in main threa